In [1]:
from datasets import load_dataset

# Load the Cakiki/Rosetta Code dataset
allData = load_dataset("cakiki/rosetta-code")

languages = ['Go','Java','JavaScript','PHP','Python','Ruby', 'Ada']
trainData = allData['train']
partDict = trainData.filter(lambda example: example['language_name']in languages)
print(len(partDict))

Found cached dataset parquet (/root/.cache/huggingface/datasets/cakiki___parquet/cakiki--rosetta-code-c68b129ca95ba990/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/cakiki___parquet/cakiki--rosetta-code-c68b129ca95ba990/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-425266b0b761cb2c.arrow


6273


# Train and test knn using embeddings extracted from BERT

In [19]:
import torch
from clustering import KNN

## load the embeddings from saved file and split it.
saved_emb = torch.load('bert_6273.pt')
saved_emb = saved_emb.cpu()
saved_emb.shape
train, test = torch.split(saved_emb,[6000, 273])

## map the labels from string to number, eg. java->0
labels = partDict['language_name']
lang_dict = dict(zip(set(partDict['language_name']), [i for i in range(len(set(partDict['language_name'])))]))
num_labels = [lang_dict[label] for label in labels]
# split the labels
train_label = torch.tensor(num_labels[:6000])
test_label = torch.tensor(num_labels[6000:])
print(len(train), len(test), len(train_label), len(test_label))


knn = KNN(train, train_label)
predicted = knn(test)

total_correct = (predicted == test_label).sum().item()

# Calculate the accuracy for this epoch
accuracy = 100 * total_correct / test_label.size(0)
print('Accuracy:  {:.2f}%'.format(accuracy))

6000 273 6000 273


# Train and test knn using embeddings extracted from GPT